# Unsteady flow induced by a moving wall

## Introduction and presentation of the problem

 ### Objective

 Simulate the transient flow around a continuous moving piston, using the characteristics method, without focusing of compression waves.

## Method of Characteristics

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize    as scopt
from aerokit.common import defaultgas
%matplotlib inline
#
#plt.rc('text', usetex=True)
sty_carac = { 'color': 'orange', 'linewidth': 2 }
sty_wall  = { 'color': 'black',  'linewidth': 3 }
sty_flow  = { 'color': 'green',  'linewidth': 3 }
sty_text  = { 'fontsize': 14 }

def fig(zoom=1):
    global ax
    fig = plt.figure(figsize=(15*zoom,15*zoom), facecolor='white')
    ax  = fig.add_subplot(111) 
    #plt.axis([xneg, length])
    # ax.set(aspect="equal")
    ax.axis('on')
    ax.set_facecolor('white') # dependt on backend
    ax.set_xlim(xmin, xmax) #, xlim=[xneg, 4*length], ylim=[-.1, ymax])
    ax.set_ylim(tmin, tmax) #, xlim=[xneg, 4*length], ylim=[-.1, ymax])
    
    
def plot_geom():
    t = np.linspace(tmin, tmax, 101)
    plt.fill(np.concatenate(([xmin, xmin], xpiston(t))), np.concatenate(([tmax, tmin], t)),facecolor='lightgray', alpha=1., zorder=100)
    plt.plot(xpiston(t), t, zorder=101, **sty_wall)
    plt.text(xmin + 0.1, tmax - 0.1, r'$a_0={:.2f}\ m/s$'.format(a0), zorder=101, backgroundcolor='white', **sty_text)
    plt.tight_layout()
    plt.xlabel("x [m]")
    plt.ylabel("t [s]")

We define the piston position and speed according to the time $ u (t) $ and $ v (t) $

In [ ]:
# definition of problem parameters
gam  = 1.4 ; defaultgas.set_gamma(gam)
a0 = 300.
tmax = 3.
tmin = 0.
xmin = -10
xmax = 0.5*tmax*a0 # 

# function to plot the geometry
# 
xpistonmax = 50.
def xpiston(t):
    return xpistonmax*np.exp(-(t/.5-3)**2)

# test de la fonction de tracé   
fig(zoom=.5); plot_geom()

## Faisceau de compression
La vitesse du son au niveau du piston est calculée à partir de l'invariant des C- venant de la zone uniforme au repos: $u_p - \frac{2a_p}{\gamma-1} = u_0 - \frac{2a_0}{\gamma-1} \Rightarrow a_p += a_0 + \frac{\gamma-1}{2}u_0$

La pente des C+ est donnée par $\frac{dt}{dx}=\frac{1}{u_p+a_p}$

In [ ]:
# Piston velocity
def u_piston(t):
    return -xpistonmax*np.exp(-(t/0.5-3)**2)*4*(t/0.5 - 3)

# speed of sound at piston boundary
def a_piston(t):
    # invariant of C- coming from uniform zone: u-2a/(gam-1)=cste
    return a0 + u_piston(t)*(gam-1.)/2.

t = np.linspace(tmin, tmax, 500)
print(f"intial speed of sound: {a0:.2f} m/s")
print(f"Maximum piston speed: {np.max(u_piston(t)):.2f} m/s")

# Function to plot the C+ from the piston boundary
def plot_CP_piston(t): 
    for tp in t:
        ap = a_piston(tp)
        slope = 1. / (u_piston(tp) + ap)
        plt.plot([xpiston(tp), xpiston(tp)+(tmax-tp)*1/slope], [tp, tmax], **sty_carac)
    
def plot_CP(n=51): 
    plot_CP_piston(np.linspace(tmin, tmax, n))

fig(zoom = 0.5)
plot_CP()
plot_geom()


## integration trajectoire et C-

On veut calculer les propriétés du fluide à un point $M(x,t)$ donné. Deux invariants arrivent à ce point.

- invariant C-, venant de la zone uniforme: $u_M - \frac{2a_M}{\gamma-1} = u_0 - \frac{2a_0}{\gamma-1}$
- invariant C+, venant du piston: $u_M + \frac{2a_M}{\gamma-1} = u_p + \frac{2a_p}{\gamma-1}$
On commence le calcul itératif de $u_M$ par initialisant la vitesse $u_M$ et la vitesse du point du piston qui intersecte la C+.
Le calcul itératif consuste à calculer la vitesse du son au point $M(x,t)$ avec la formule de l'invariant de la C-, pour ensuite calculer la pente de la C+. Ensuite on calculte le point d'intersection de la C+ avec le piston et on calcule la vitesse du piston en ce point. On mmet ensuite a jour la vitesse au point $M(x,t)$. Le calcul continue jusqu'à ce que la vitesse converge ou le nombre maximal d'itérations est atteint.

En connaissant la vitesse du fluide en tout point et moment, on peut ensuite intégrer la trajectoire d'une particule de fluide.

In [52]:
# u velocity at a given point        
def u_vel(x, t, tol=1e-6, max_iter=100):
    # Initial guess for u at point (x,t) and up at point on the wall where the C+ originates
    a = lambda u: a0 + (gam-1)/2*u # C- relation
    u = 0    
    err = 1.
    # iterate while the solution has not converged
    # print(f"calculating u_vel at x={x}, t={t:.3f}")
    iter = 0        # Iteration counter
    while err > tol and iter <= max_iter:
        # find the slope of the C+
        invslope = u+a(u)
        t_guess = t-x/invslope
        # solve intersection with piston: x - (t-tp)/slope = xpiston(tp)
        tpiston = scopt.fsolve(lambda tp: x-xpiston(tp)-(t-tp)*invslope, t_guess)
        # Update u based on piston properties
        up = u_piston(tpiston)  # calculate the flow properties at the piston at the calculated time tpiston 
        err = np.abs(u-up)      # calculate the error
        #u = up
        u = 0.5*(u+up)       # under-relaxation
        iter += 1
    if iter > max_iter:
        print(f"u_vel did not converge at x={x}, t={t:.3f}")
        # raise RuntimeError(f"u_vel did not converge at x={x}, t={t}")
        return u
    return u

def flow_prop(x, t):
    u = u_vel(x, t)
    # from C-
    a = a0 + (gam-1.)/2. * u
    prop = {'vel': u, 'C+': u+a, 'C-': u-a}
    return prop

def integ(x, t, ctype, smin, smax, npts=100):
    def step(x0, t0, dt, u):
        return x0+dt*u, t0+dt
    trajx = np.zeros(npts+1)
    trajt = np.zeros(npts+1)
    nforw = int(npts * smax/(smax-smin))
    nback = npts-nforw
    #print(smin, smax, nforw, nback)
    sx = x
    st = t
    trajx[nback] = sx ; trajt[nback] = st
    # backward
    if nback>0:
        dt = smin/nback
        # print(f"dt: {dt:.4f}")
        for i in range(nback):
            px, pt = step(sx, st, .5*dt, flow_prop(sx, st)[ctype])    # RK2 / predictor
            sx, st = step(sx, st,    dt, flow_prop(px, pt)[ctype])    # RK2 / final step
            trajx[nback-i-1] = sx ; trajt[nback-i-1] = st
    # forward
    if nforw>0:
        dt = smax/nforw
        # print(f"dt: {dt:.4f}")
        sx = x
        st = t
        for i in range(nforw):
            px, pt = step(sx, st, .5*dt, flow_prop(sx, st)[ctype])    # RK2 / predictor
            sx, st = step(sx, st,    dt, flow_prop(px, pt)[ctype])    # RK2 / final step
            trajx[nback+i+1] = sx ; trajt[nback+i+1] = st
    return trajx, trajt
    

In [ ]:
def plot_CM(n=31, length=-1.): 
    t = np.linspace(tmin, tmax, n)
    for tp in t:
        xcm, tcm = integ(xpiston(tp), tp, 'C-', length, 0.)
        plt.plot(xcm, tcm, **sty_carac)

        
# fluid particle trajectories
Paths = []
for i, (x0, t0) in enumerate([(25, .25), (35, .25), (100, .25), (270, 2), (280, 2)]):
    print(f"Calculating trajectory {i}: x0={x0}, t0={t0}")
    Paths.append(integ(x0, t0, 'vel', 0., 4.))
 
fig(zoom = 0.5)
print("Calculating C+ for plot")
plot_CP()
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()

fig(zoom = 0.5)
print("Calculating C- for plot")
plot_CM(length=-3)
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()

fig(zoom = 0.5)
print("Calculating C+ for plot")
plot_CP()
print("Calculating C- for plot")
plot_CM(length=-3)
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()


## Détente progressive

In [ ]:
# definition of problem parameters
# 
tmax = 3.
tmin = 0.
xmin = -200
xmax = 0.6*tmax*a0 # 


# function to plot the geometry
# 
def xpiston(t):
    slope = -180  # Define the slope for the linear portion
    return np.where(
        t < 0.75, 
        0.,  # Piston is stationary for t < 0.75
        np.where(
            t < 1.5,
            slope*(t-0.75)**2 / (2*(1.5-0.75)),  # Quadratic behavior between 0.75 and 1.5
            slope*(t-1.5) + slope*(1.5-0.75)**2 / (2*(1.5-0.75))  # Linear, continuous at t = 1.5
        )
    )

def u_piston(t):
    slope = -180  # Slope of the linear portion
    
    return np.where(
        t < 0.75,
        0.,  # Velocity is 0 for t < 0.75
        np.where(
            t < 1.5,
            slope*(t-0.75) / (1.5-0.75),  # Derivative of the quadratic portion
            slope  # Constant velocity for the linear portion
        )
    )

t = np.linspace(tmin, tmax, 500)
print(f"vitesse du son: {a0:.2f} m/s")
print(f"vitesse maximum du piston: {np.min(u_piston(t)):.2f} m/s")

# fluid particle trajectories
x0 = 50 ; t0 = .25
print(f"Calculating tajectory 1: x0={x0}, t0={t0}")
CTx1, CTt1 = integ(x0, t0, 'vel', 0., 4.)
x0 = 200 ; t0 = .25
print(f"Calculating tajectory 2: x0={x0}, t0={t0}")
CTx2, CTt2 = integ(x0, t0, 'vel', 0., 4.)
Paths = []
for i, (x0, t0) in enumerate([(50, .25), (60, .25), (150, .25)]):
    print(f"Calculating trajectory {i}: x0={x0}, t0={t0}")
    Paths.append(integ(x0, t0, 'vel', 0., 4.))

fig(zoom = 0.5)
print("Calculating C+ for plot")
plot_CP()
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()

fig(zoom = 0.5)
print("Calculating C- for plot")
plot_CM(length=-3)
plot_CP_piston([0.75, 1.5])
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()


## Compression

In [ ]:
# definition of problem parameters
# 
tmax = 2.5
tmin = 0.
xmin = -10
xmax = 0.5*tmax*a0 # 

# function to plot the geometry
# 
def xpiston(t):
    slope = 150  # Define the slope for the linear portion
    return np.where(
        t < 0.75, 
        0.,  # Piston is stationary for t < 0.75
        np.where(
            t < 1.5,
            slope*(t-0.75)**2 / (2*(1.5-0.75)),  # Quadratic behavior between 0.75 and 1.5
            slope*(t-1.5) + slope*(1.5-0.75)**2 / (2*(1.5-0.75))  # Linear, continuous at t = 1.5
        )
    )

def u_piston(t):
    slope = 150  # Slope of the linear portion    
    return np.where(
        t < 0.75,
        0.,  # Velocity is 0 for t < 0.75
        np.where(
            t < 1.5,
            slope*(t-0.75)/(1.5-0.75),  # Derivative of the quadratic portion
            slope  # Constant velocity for the linear portion
        )
    )

t = np.linspace(tmin, tmax, 500)
print(f"vitesse du son: {a0:.2f} m/s")
print(f"vitesse maximum du piston: {np.max(u_piston(t)):.2f} m/s")

# fluid particle trajectories
Paths = []
for i, (x0, t0) in enumerate([(50, .25), (60, .25), (200, .25)]):
    print(f"Calculating trajectory {i}: x0={x0}, t0={t0}")
    Paths.append(integ(x0, t0, 'vel', 0., 4.))
 
fig(zoom = 0.5)
print("Calculating C+ for plot")
plot_CP()
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()

fig(zoom = 0.5)
print("Calculating C- for plot")
plot_CM(length=-3)
plot_CP_piston([0.75, 1.5])
for x, t in Paths:
    plt.plot(x, t, **sty_flow)
plot_geom()


**credits:** Solim ROVERA, J. GRESSIER